<a href="https://colab.research.google.com/github/tap-o-chek/Multiple-processing-course/blob/main/MP_Lesson_4_%D0%96%D0%B8%D0%B4%D0%B5%D0%BB%D0%B5%D0%B2%D0%B0_%D0%A1%D0%BE%D1%84%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Использовать MRJob

In [ ]:
!pip install mrjob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 6.6 MB/s eta 0:00:00


полезно почитать - https://coderlessons.com/tutorials/bolshie-dannye-i-analitika/izuchit-kartu-umenshit/mapreduce-kratkoe-rukovodstvo

**Упражнение 1.**
Дан файл с числами (несколько строк, на которых через пробел расположены числа). Посчитать сумму всех чисел в файле (построчно через MRJob).

Пример:

1 2 3

4 5

6

Вывод:

21

In [ ]:
!python task1.py input_1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task1.root.20240321.181952.337864
Running step 1 of 1...
job output is in /tmp/task1.root.20240321.181952.337864/output
Streaming final output from /tmp/task1.root.20240321.181952.337864/output...
null	21
Removing temp directory /tmp/task1.root.20240321.181952.337864...


Содержимое файла
```
from mrjob.job import MRJob

class MRSumNumbers(MRJob):

    def mapper(self, _, line):
        numbers = map(int, line.strip().split())
        for number in numbers:
            yield None, number # отправляем каждое число как ключ и его значение

    def reducer(self, _, numbers):
        total_sum = sum(numbers)
        yield None, total_sum


if __name__ == '__main__':
    MRSumNumbers.run()
```



**Упражнение 2.**
Дан файл с числами (несколько строк, на которых через пробел расположены числа). Посчитать сумму чисел в файле в каждой строке, а еще и их среднее (построчно через MRJob).

Пример:

1 2 3

4 5

6

Вывод:

6, 2

9, 4.5

6, 6

In [ ]:
!python task2.py input_2.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task2.root.20240321.181952.754472
Running step 1 of 1...
job output is in /tmp/task2.root.20240321.181952.754472/output
Streaming final output from /tmp/task2.root.20240321.181952.754472/output...
6.0	2.0
9.0	4.5
6.0	6.0
Removing temp directory /tmp/task2.root.20240321.181952.754472...


Содержимое файла
```
from mrjob.job import MRJob


class MRSumAndAverage(MRJob):

    def mapper(self, _, line):
        numbers = list(map(float, line.split()))
        yield None, (sum(numbers), len(numbers))

    def reducer(self, _, values):
        for summ, num_count in values:
            average = summ / num_count
            yield summ, average


if __name__ == '__main__':
    MRSumAndAverage.run()
```



**Упражнение 3.**
Дан файл со стоками, в каждой строке слова через пробел. Посчитать самую частую букву.

Пример:

aaa bb c

aaa bb

aaa

Вывод:

9 a

In [ ]:
!python task3.py input_3.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task3.root.20240321.181953.371177
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/task3.root.20240321.181953.371177/output
Streaming final output from /tmp/task3.root.20240321.181953.371177/output...
9	"a"
Removing temp directory /tmp/task3.root.20240321.181953.371177...


Содержимое файла
```
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
import re

class MostCommonLetter(MRJob):

    def mapper(self, _, line):
        letters = re.findall('[a-zA-Z]', line.lower())
        for letter in letters:
            yield letter, 1

    def combiner(self, letter, counts):
        yield letter, sum(counts)

    def reducer(self, letter, counts):
        yield None, (sum(counts), letter)

    def second_reducer(self, _, counts):
        most_common_count, most_common_letter = max(counts)
        yield most_common_count, most_common_letter

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(reducer=self.second_reducer)
        ]

if __name__ == '__main__':
    MostCommonLetter.run()
```



**Упражнение 4.**
Есть множество записей. Каждая запись содержит поле F и производное число категорий G={G1, G2,...}
Посчитать общее число уникальных значений поля F для каждой категории.

Пример:

Record1: F = 1, G = {a, b}

Record2: F = 2, G = {a, d, e}

Record3: F = 1, G = {b}

Record4: F = 13, G = {a, b}

Result:

a -> 3 // F =1, F=2, F=3

b -> 2 // F =1, F=3

d -> 1 // F =2

e -> 1 // F =2

In [ ]:
!python task4.py input_4.txt

Traceback (most recent call last):
  File "/content/task4.py", line 38, in <module>
    UniqueFCount.run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 634, in run_job
    with self.make_runner() as runner:
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 704, in make_runner
    kwargs = self._runner_kwargs()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 734, in _runner_kwargs
    kwargs = dict(steps=self._steps_desc(), **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 916, in _steps_desc
    for step_num, step in enumerate(self.steps()):
  File "/content/task4.py", line 31, in steps
    self.mr(mapper=self.mapper,
AttributeError: 'UniqueFCount' object has no attribute 'mr'


Содержимое файла
```
from mrjob.job import MRJob
from mrjob.step import MRStep

class UniqueFValuesByCategory(MRJob):

    def mapper(self, _, line):
        record = line.strip().split(":")
        F = record[1].strip().split('=')[1]
        categories = record[2].strip().split('=')[1].strip('{}').split(', ')
        
        for category in categories:
            yield category, int(F)

    def reducer(self, category, values):
        unique_values = set(values)
        yield category, (sum(values), sum(values) / len(values))

if __name__ == '__main__':
    UniqueFValuesByCategory.run()
```



**Упражнение 5.**
Есть множество кортежей объектов. Для каждой возможной пары объектов посчитать число картежей, где они встретились вместе. Если число объектов N, то N*N объектов будет обработано. Если N*N небольшое, то можно построить матрицу в памяти и реализация довольно простая.

In [ ]:
!python task5.py input_5.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/task5.root.20240321.192643.059829
Running step 1 of 1...
job output is in /tmp/task5.root.20240321.192643.059829/output
Streaming final output from /tmp/task5.root.20240321.192643.059829/output...
["B", "D"]	3
["C", "C"]	1
["C", "D"]	3
["D", "B"]	1
["A", "B"]	3
["A", "C"]	4
["A", "D"]	3
["B", "B"]	1
["B", "C"]	3
Removing temp directory /tmp/task5.root.20240321.192643.059829...


Содержимое файла
```
from mrjob.job import MRJob
from collections import defaultdict

class MRCountTupleOccurrences(MRJob):
    
    def mapper(self, _, line):
        tuple_elements = line.strip().split()
        pairs = [(tuple_elements[i], tuple_elements[j]) for i in range(len(tuple_elements))
                 for j in range(i+1, len(tuple_elements))]
        for pair in pairs:
            yield pair, 1

    def reducer(self, pair, counts):
        yield pair, sum(counts)

if __name__ == '__main__':
    MRCountTupleOccurrences.run()
```



Подсказки к 4 и 5 тут - https://www.slideshare.net/Technopark/lecture-05-48215737